# Tablet Fiyat/Performans Analizi

Mağazaya yeni gelen tabletleri fiyat/performans özelliklerine göre sınıflayabilen bir makine öğrenmesi yapmamız istenmektedir. Bunu keşifçi veri analizi, ön işleme ve tahmine dayalı analitik makine öğrenmesi olmak üzere 3 aşamada yapacağız.

Bunlara başlamadan önce ihtiyacımız olan kütüphaneleri ve veri setini ekleyelim.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import missingno
from sklearn import preprocessing
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

İleride yapacağımız değişiklerin orijinal veri setini etkilememesi için veri setinin kopyası üzerinde çalışacağız.

In [ ]:
tablet = pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")
df = tablet.copy() 

# **Keşifçi Veri Analizi**

Veriyi tanımak için ilk olarak veri setinin ilk 5 gözlemine bakalım.

In [ ]:
df.head()

Değişken ve gözlem sayısına bakalım.

In [ ]:
df.shape

Veri setinin içinde 2000 adet tablet ve 20 adet değişken var.

Değişkenlerin tiplerine bakalım.

In [ ]:
df.dtypes

Eksik gözlem var mı, eğer varsa kaç adet ?

In [ ]:
df.isna().sum()

Ürünlerden 5 tanesinin ön kamera,12 tanesinin Ram bilgisi eksik. 
Eksik gözlemler ile ikinci aşamada ilgileneceğiz. Şimdilik devam edelim.

Hangi renkte tabletler var ?

In [ ]:
print(df["Renk"].unique())

Fiyat aralıkları istenildiği gibi girilmiş mi ?

In [ ]:
print(df["FiyatAraligi"].unique())

Çift hat özelliği olan tabletlere bakmak istesek ?

In [ ]:
df[df["CiftHat"] == "Var"]

Tabletlerin batarya gücüne fiyat aralıkları ile birlikte bakabilir miyiz ?

In [ ]:
df.groupby("FiyatAraligi").describe()["BataryaGucu"]

Peki tabletlerin WiFi, Bluetooth, dokunmatik ve 4G özelliklerine göre batarya güçlerinin ortalamasına baksak ?

In [ ]:
pd.pivot_table(df, values = "BataryaGucu", index = ["WiFi", "Bluetooth", "Dokunmatik", "4G"],columns = ["FiyatAraligi"], aggfunc = np.mean)

Batarya ömrünü de eklemek istesek ?

In [ ]:
pd.pivot_table(df, values = ["BataryaGucu", "BataryaOmru"], index = ["WiFi", "Bluetooth", "Dokunmatik", "4G"],columns = ["FiyatAraligi"], aggfunc = np.mean)

Bu gruplamaya renkleri de ekleyelim.

In [ ]:
pd.pivot_table(df, values = ["BataryaGucu", "BataryaOmru"], index = ["WiFi", "Bluetooth", "Dokunmatik", "4G","Renk"],columns = ["FiyatAraligi"], aggfunc = np.mean)

Tabletlerin çekirdek sayılarının dağılımına grafik ile daha iyi bakalım.

In [ ]:
sns.distplot(df["CekirdekSayisi"], bins=8, color="red");

Tabletlerin renk dağılımı nasıl ?

In [ ]:
sns.countplot(df["Renk"]);

Tabletlerin çözünürlükleri nasıl, grafik ile görebilir miyiz ?

In [ ]:
sns.scatterplot(x = "CozunurlukYükseklik", y = "CozunurlukGenislik", data = df);

Fazla anlaşılır olmadı gibi, jointplot ile baksak ?

In [ ]:
sns.jointplot(x = df["CozunurlukYükseklik"], y = df["CozunurlukGenislik"], kind = "kde", color = "turquoise");

Tabletlerin çözünürlüklerini fiyat aralığına göre ayırsak ?

In [ ]:
sns.scatterplot(x = "CozunurlukYükseklik", y = "CozunurlukGenislik", hue = "FiyatAraligi",  data = df);

Daha anlaşılır olması için bir çizgi çizsek ? 

In [ ]:
sns.lmplot(x = "CozunurlukYükseklik", y = "CozunurlukGenislik", hue = "FiyatAraligi",  data = df);

Fiyat aralığına göre ürünlerin RAM'lerinin dağılımına violinplot ile bakabilir miyiz ?

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "RAM",  data = df);

Her bir fiyat aralığının ürün sayıları birbirine yakın mı ?

In [ ]:
sns.countplot(df["FiyatAraligi"])

Grafiğe bakarsak her fiyat aralığının 500 ürünü var. Elimizdeki veri seti dengeli.

Değerler arasında korelasyon var mı? Bunu yorumlamak için önce korelasyon matrisi çizdirelim.

In [ ]:
corr = df.corr()
corr

Çok fazla değişkenimiz var ve korelasyonu bu şekildeyken yorumlamak oldukça zorlayıcı görünüyor. Isı haritası çizdirip okunmasını daha kolay hale getirelim.

In [ ]:
corr=df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values);

Çoğu değer için korelasyon neredeyse yok denecek kadar zayıf. CozunurlukYükseklik ve CozunurlukGenislik arasında pozitif fakat çok güçlü olmayan bir korelasyon var. Değişkenler arasında en güçlü ve pozitif korelasyon ise ArkaKameraMP ve OnKameraMP arasında.

Arka kamera ile ön kamera arasındaki korelasyona bir de jointplot ile bakalım.

In [ ]:
with sns.axes_style('dark'):
    sns.jointplot("ArkaKameraMP", "OnKameraMP", data = df, kind = "hex", color = "green")

Bu iki değer arasındaki korelasyon katsayısını da yazdıralım.

In [ ]:
df.corr()["ArkaKameraMP"]["OnKameraMP"]

Veri setindeki sayısal değişkenlere bakalım.

In [ ]:
df.describe()

Tabletlerin ortalama batarya ömrü ne kadar ? 

In [ ]:
df["BataryaOmru"].mean()

Fiyatı ucuz olan tabletlerin ortalama batarya gücü, mikro işlemci hızı, dahili belleği ve Ram'i ne kadar ?

In [ ]:
df[df["FiyatAraligi"] == "Ucuz"][["BataryaGucu","MikroislemciHizi","DahiliBellek","RAM"]].mean()


Fiyatı çok ucuz olan tabletlerde en iyi kamera kalitesi kaç MP ?

In [ ]:
df[df["FiyatAraligi"] == "Çok Ucuz"][["OnKameraMP","ArkaKameraMP"]].max()

En hafif tabletin ağırlığı ne kadar ?

In [ ]:
df["Agirlik"].min()

Tabletlerin fiyat aralıklarına göre Ram'leri nasıl ?

In [ ]:
df.groupby("FiyatAraligi")["RAM"].mean()

Pahalı tabletler çok ucuz tabletlerin neredeyse 4 katı Ram'e sahip.

# **Veri Ön İşleme**

Eksik verilere bu aşamada döneceğimizi söylemiştik. Ön kamerada 5, RAM'de 12 adet eksik veri vardı.

Bu eksik veriler birbirleriyle ilişkili mi ?

In [ ]:
missingno.matrix(df,figsize=(12, 10));

Bir de ısı haritası çizdirelim.

In [ ]:
missingno.heatmap(df, figsize= (6,6));

Bembeyaz ısı haritasına bakarak eksik verilerin arasında hiç bir ilişki olmadığını söyleyebiliriz.

* İlk önce ön kameradaki eksik veriler ile ilgilenelim.

Neden ön kamera bilgisi girilmemiştir?

*Tablet çok ucuz bir tablet olup ön kamerası olamayabilir, bu nedenle de ön kamera bilgisi girilmemiş olabilir mi?*

Bunun doğruluğunu inceleyelim. Eğer tabletler çok ucuz oldukları için ön kameraları yoksa mantıken tabletlerin diğer özellikleri de ortalamanın altındadır. 
Önceklikle ön kamera bilgisi eksik olanlar tabletlere bakalım.

Ön kamera verisi girilmeyen tabletlerin bilgilerine daha kolay ulaşmak için tabloyu eok değişkenine atayalım.

In [ ]:
eok = df[df["OnKameraMP"].isnull()]
eok

Düşündüğümüz gibi tabletlerin hepsi çok ucuz kategorisinde.

Diğer özelliklere de bakalım. Örneğin tabletlerin RAM'leri ortalamanın aşağısında mı ?

In [ ]:
print("Ortalama RAM: ", df["RAM"].mean(),
      "\nBu tabletlerdeki ortalama RAM:", eok["RAM"].mean())

Evet RAM'ler ortalamanın aşağısında.

Peki bataryaları nasıl ?

In [ ]:
print("Ortalama batarya gücü: ", df["BataryaGucu"].mean(),", batarya ömrü: ",df["BataryaOmru"].mean() , 
      "\nBu tabletlerdeki ortalama batarya gücü:", eok["BataryaGucu"].mean(), ", batarya ömrü: ", eok["BataryaOmru"].mean())

İki değer de ortalamanın aşağısında, yani bu tabletlerin bataryaları genele oranla daha kötü.

Kararımızı vermeden önce son olarak arka kameraya da bakalım.

In [ ]:
print("Ortalama arka kamera MP'si: ", df["ArkaKameraMP"].mean(), 
      "\nBu tabletlerdeki ortalama arka kamera MP'si:", eok["ArkaKameraMP"].mean())

Arka kamera ortalamanın altında olsa da diğer özellikler kadar kötü değil.

Bunlara bakarak bu tabletlerin çok kaliteli olmayan tabletler olduğunu ve ön kameralarının olmadığını söyleyebiliriz.O zaman ön kameralar için 0 yazmamızda bir sakınca yok.

Bunun için önce ön kamera bilgileri girilmemiş tabletlerin indislerini bir değişkene atayalım.

In [ ]:
eokT = eok["OnKameraMP"].index
eokT

Daha sonra .loc ile indislerdeki ön kamera bilgisine 0 girelim.

In [ ]:
df.loc[eokT ,"OnKameraMP"] = 0

Ön kamera için 0 girerek eksik verileri doldurduk. Tedbir amaçlı kontrol edelim.

In [ ]:
df.isna().sum()["OnKameraMP"]

In [ ]:
df["OnKameraMP"].unique()

Tamamdır.

* Şimdi RAM'e geçebiliriz.

*RAM bilgisi girilmemesinin nedeni dikkatsizlik olabilir.*

İlk olarak Ram bilgisi girilmemiş olan tabletlere bakalım.

In [ ]:
df[df["RAM"].isnull()]

Bu tabletlerin hepsi pahalı kategorisinde.

*Hepsi pahalı tabletlerse kaliteli tabletlerdir ve  RAM'lerinin de iyi olduğunu varsayabiliriz. O zaman girilmemesinin nedeni büyük ihtimalle dikkatsizliktir.*

Tabletlerin kaliteli olduğu düşüncesini desteklemek için birkaç özelliğe daha bakalım.

In [ ]:
print("Ortalama batarya gücü: ", df["BataryaGucu"].mean(),", batarya ömrü: ",df["BataryaOmru"].mean() , 
      "\nBu tabletlerdeki ortalama batarya gücü:", df[df["RAM"].isnull()]["BataryaGucu"].mean(), ", batarya ömrü: ", df[df["RAM"].isnull()]["BataryaOmru"].mean())

Bataryaları ortalama üstünde.

In [ ]:
print("Ortalama mikro işlemci hızı: ", df["MikroislemciHizi"].mean(), 
      "\nBu tabletlerdeki ortalama mikro işlemci hızı:", df[df["RAM"].isnull()]["MikroislemciHizi"].mean())

Mikro işlemci hızları ortalamaya yakın.

In [ ]:
print("Ortalama dahili bellek: ", df["DahiliBellek"].mean(), 
      "\nBu tabletlerdeki ortalama dahili bellek:", df[df["RAM"].isnull()]["DahiliBellek"].mean())

Dahili bellek de ortalamanın üstünde.Bunlara bakarak bu tabletlerin kaliteli tabletler olduğunu söyleyebiliriz.

Bu tabletlerin RAM'lerini Pahalı kategorisindeki tabletlerin ortalama RAM değeri ile dolduralım.

Pahalı tabletlerin ortalama RAM'i ne kadar ?

In [ ]:
df[df["FiyatAraligi"] == "Pahalı"][["RAM"]].mean()

RAM bilgisi olmayan tabletlerin indislerini de bir değişkene atayalım.

In [ ]:
erT = df[df["RAM"].isnull()]["RAM"].index
erT

Ön kameradaki gibi .loc ile indislerdeki RAM bilgilerine pahalı tabletlerin ortalama RAM bilgilerini girelim.

In [ ]:
df.loc[erT, "RAM"] = 3449.35041

Tedbir amaçlı bunu da kontrol edelim.

In [ ]:
df["RAM"].isna().sum()

In [ ]:
print(df["RAM"].unique())

Böylelikle eksik verileri doldurmuş olduk.

Şimdi kategorik verileri sayısallaştırmaya geçelim.

4G, WiFi gibi iki değere sahip değişkenleri label encoding ile 0,1 olarak sayısallaştıracağız. Var için 0, yok için 1 olacak.

Fiyat aralığı sıralanabilir bir özellik olduğu için fiyat aralığını da label encoder ile sayısallaştıracağız.

Label encoder değişkenini oluşturalım ve label encoding yapalım.

In [ ]:
label_encoder = preprocessing.LabelEncoder()

In [ ]:
df['Bluetooth'] = label_encoder.fit_transform(df['Bluetooth']) 
df['CiftHat'] = label_encoder.fit_transform(df['CiftHat'])
df['4G'] = label_encoder.fit_transform(df['4G'])
df['3G'] = label_encoder.fit_transform(df['3G'])
df['Dokunmatik'] = label_encoder.fit_transform(df['Dokunmatik'])
df['WiFi'] = label_encoder.fit_transform(df['WiFi'])
df['FiyatAraligi'] = label_encoder.fit_transform(df['FiyatAraligi']) # Normal: 0, Pahalı: 1, Ucuz: 2, Çok Ucuz: 3 oldu.
df.head()

Renkler ise sıralamaz, bu yüzden renkler için label kullanamayız. Onun yerine get_dummies kullanacağız.

In [ ]:
df['Renk'] = pd.Categorical(df['Renk'])
dfDummiesR = pd.get_dummies(df['Renk'])
dfDummiesR

Dummysini yarattığımız sınıf ile onun dummylerinden biri silinmelidir. Sınıf artık dummyleri olduğu için gereksizdir, diğer dummylere bakarak da eksik olan hakkında çıkarımda bulunabilir.

In [ ]:
df = pd.concat([df, dfDummiesR], axis=1)
df.drop(["Renk"], axis = 1, inplace = True)
df.drop(["Yeşil"], axis = 1, inplace = True)

Tablonun son haline bakıp üçüncü aşamaya geçelim.

In [ ]:
df.head()

# **Tahmine Dayalı Analitik Makine Öğrenmesi**

Veri çerçevesini bağımlı ve bağımsız değişkenler olarak bölütleyelim.

Tabletlerin özelliklerine göre uygun fiyat aralığını bulmamız gerekli.Bu durumda bağımlı değişken fiyat aralığı, bağımsız değişkenler de tabletlerin özellikleri olacaktır. X bağımsız değişkenler, Y bağımlı değişkendir.

In [ ]:
X = df.drop((["FiyatAraligi"]), axis=1)
Y = df["FiyatAraligi"]

Train_test_split kullanarak bağımlı ve bağımsız değişkenlerden test ve train olmak üzere 4 parça oluşturalım.

Bağımlı ve bağımsız değişkenlerin %75'i eğitim ve %25'i test için kullanılacaktır.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 1/4, random_state = 20, shuffle=1)

Eğitim kümesi ile GaussianNB, DecisionTree ve KNN modellerini eğitelim.

GasussianNB modelini inşa edip bağımlı ve bağımsız değişkenlerden oluşan eğitim verileri ile eğitelim.Daha sonra bağımlı test değişkenini tahmin ettirelim ve y_pred_GNB değişkenine atayalım.

In [ ]:
model_GNB = GaussianNB()
model_GNB.fit(X_train, y_train)
y_pred_GNB = model_GNB.predict(X_test)

DesicionTree modelini inşa edip bağımlı ve bağımsız değişkenlerden oluşan eğitim verileri ile eğitelim. Daha sonra bağımlı test değişkenini tahmin ettirelim ve y_pred_DT değişkenine atayalım.

In [ ]:
module_DT = DecisionTreeClassifier()
module_DT = module_DT.fit(X_train,y_train)
y_pred_DT = module_DT.predict(X_test)

DesicionTree modelinindeki criterion parametresinin değerini varsayılan değer yerine entropy olarak değiştirdiğimizde başarı skorunun artacağı varsayılıyor. O zaman parametrenin değerini değiştirip bir DecisionTree modeli daha eğitelim ve başarı skorlarını karşılaştıralım. Eğer gerçekten criterion = entropy olması başarıyı arttırıyorsa bu modeli kullanalım.

In [ ]:
module_DTE = DecisionTreeClassifier(criterion = "entropy")
module_DTE = module_DTE.fit(X_train,y_train)
y_pred_DTE = module_DTE.predict(X_test)

Başarı skorlarını karşılaştıralım.

In [ ]:
accuracy_DT = accuracy_score(y_test, y_pred_DT)
accuracy_DTE = accuracy_score(y_test, y_pred_DTE)
print("Varsayılan criterion ile başarı skoru: ", accuracy_DT, 
      " \ncriterion = entropy ile başarı skoru: ", accuracy_DTE)

Evet, criterion = entropy olduğunda başarı skoru 0.038 artıyor. Bu durumda criterion = entropy olan module_DTE modelini kullanalım.

KNN modelini inşa edip bağımlı ve bağımsız değişkenlerden oluşan eğitim verileri ile eğitelim. Modeli inşa ederken 2’den 15’e kadar olan her sayıyı komşu sayısı olarak deneyelim ve başarı skorunu etkiliyor mu bakmak için yazdıralım. Bunun için her seferinde bağımlı test değişkenini tahmin ettirelim ve y_pred_KNN değişkenine atayalım. Her değer için başarı skorunu da y_pred_KNN_Score adlı bir listeye kaydedelim.

In [ ]:
y_pred_KNN_Score = []
for k in range(2,15):
    module_KNN = KNeighborsClassifier(n_neighbors = k)
    module_KNN = module_KNN.fit(X_train, y_train)
    y_pred_KNN = module_KNN.predict(X_test)
    y_pred_KNN_Score.append(accuracy_score(y_test, y_pred_KNN))

Başarı skorlarını tuttuğumuz listeyi yazdıralım.

In [ ]:
for k in range(2,15):
    print( "Komşu sayısı" , k , "iken modelin başarı skoru :" , y_pred_KNN_Score[k- 2])
    #k 2'den başlarken liste 0'dan başlar. Bu yüzden liste elemanı için k - 2 yazıyoruz.

Her komşu sayısı için başarı skorunu bir plot çizdirerek görselleştirelim.

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(range(2, 15), y_pred_KNN_Score, color='gold', linestyle='-', marker=".",
         markerfacecolor='orange', markersize = 12)
plt.title('Komşu Sayısına Göre KNN Modelinin Başarı Skoru ')
plt.xlabel('Komşu Sayısı')
plt.ylabel('Başarı Skoru')

Model en iyi başarı skorunu komşu sayısı 13 iken veriyor. O zaman komşu sayısını 13 alarak devam edelim.

In [ ]:
module_KNN = KNeighborsClassifier(n_neighbors = 13)
module_KNN = module_KNN.fit(X_train, y_train)
y_pred_KNN = module_KNN.predict(X_test)

GaussianNB için karmaşıklık matrisini yazalım.

In [ ]:
GNB_ConMat = confusion_matrix(y_test,y_pred_GNB)
print(GNB_ConMat)

Matrisin asal köşegen üzerindeki elemanlarının toplamı doğru tahmin edilen verileri, asal köşegen üzerinde olmayan elemanların toplamı da yanlış tahmin edilen verileri gösterir. GaussianNB modeli 500 test verisinden 78 + 89 + 75 + 110 = 352 tanesini doğru, 148 tanesini yanlış tahmin etti.

DecisionTree için karmaşılık matrisini yazalım.

In [ ]:
DTE_ConMat = confusion_matrix(y_test,y_pred_DTE)
print(DTE_ConMat)

DecisionTree modeli 500 test verisinden 108 + 103 + 97 + 123 = 429 tanesini doğru, 71 tanesini yanlış tahmin etti.

KNN için karmaşıklık matrisini yazalım.

In [ ]:
KNN_ConMat = confusion_matrix(y_test,y_pred_KNN)
print(KNN_ConMat)

KNN modeli 500 test verisinden 112 + 108 + 121 + 127 = 468 tanesini doğru, 32 tanesini yanlış tahmin etti.

Modellerin doğruluk skorlarını karşılaştıralım.

In [ ]:
accuracy_GNB = accuracy_score(y_test, y_pred_GNB)
accuracy_KNN = accuracy_score(y_test, y_pred_KNN)
print("GaussianNB modelinin başarı skoru : ", accuracy_GNB, 
      "\nDecisionTree modelinin başarı skoru : ", accuracy_DTE, 
      "\nKNN modelinin başarı skoru : " , accuracy_KNN)

Doğruluk skorlarına bakarsak en doğru sonucu veren KNN modeliyken, en az doğru sonucu veren de GaussianNB modelidir.

Şimdi classification_report ile modellerin F1_Score, Precision ve Recall değerlerine bakalım.

GaussianNB modeli için sınıflandırma raporu

In [ ]:
crGNB = classification_report(y_test, y_pred_GNB)
print(crGNB)

DecisionTree modeli için sınıflandırma raporu

In [ ]:
crDTE = classification_report(y_test, y_pred_DTE)
print(crDTE)

KNN modeli için sınıflandırma raporu

In [ ]:
crKNN = classification_report(y_test, y_pred_KNN)
print(crKNN)